In [5]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-w8iafvzk
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-w8iafvzk
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=ff607a5bcf4eeb285c40ce503103bb09c8bad810c416de75058249dba4483bfc
  Stored in directory: /tmp/pip-ephem-wheel-cache-42o292xq/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [7]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [8]:
%%cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <cuda_runtime_api.h>
#include <algorithm>
#include <cassert>
#include <cstdlib>
#include <time.h>
#include <iostream>
#include "device_launch_parameters.h"
using namespace std;

#define MAX_MASK_WIDTH 10
__constant__ float M[MAX_MASK_WIDTH];


__global__ void convolution_1D_basic_kernel(int* N, int* M, int * P, int Mask_Width, int Width){
  
  int i = threadIdx.x + blockDim.x*blockIdx.x;
  int Pvalue = 0;
  int N_start_point = i - (Mask_Width/2);
  for (int j=0; j<Mask_Width; j++){
      if (N_start_point + j >= 0 && N_start_point + j < Width){
        Pvalue += N[N_start_point+j] * M[j];
      }
  }
  P[i] = Pvalue;
}

int main(){

  const int ARRAY_SIZE = 9;
  const int ARRAY_WIDTH = 9;
  const int MASK_WIDTH = 5;
  const int MASK_SIZE = 5;

  int h_N[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
  int h_M[] = {1, 1, 1, 1, 1};
  int *h_P;
  int* d_N, * d_M, * d_P;
  
  cudaMallocHost(&h_P, ARRAY_SIZE * sizeof(int));

  cudaMalloc(&d_N, ARRAY_SIZE * sizeof(int));
  cudaMalloc(&d_M, MASK_SIZE*sizeof(int));
  cudaMalloc(&d_P, ARRAY_SIZE * sizeof(int));

  cudaMemcpy(d_N, h_N, ARRAY_SIZE * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_M, h_M, MASK_SIZE * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpyToSymbol(M, h_M, MASK_WIDTH * sizeof(int));

  convolution_1D_basic_kernel<<<1, ARRAY_WIDTH>>>(d_N, d_M, d_P, MASK_WIDTH, ARRAY_WIDTH);

  cudaMemcpy(h_P, d_P, ARRAY_SIZE * sizeof(int), cudaMemcpyDeviceToHost);  
  cudaDeviceSynchronize();
  for (int i=0; i<ARRAY_SIZE; i++) cout << h_P[i] << " ";
  return 0;
}

6 10 15 20 25 30 35 30 24 
